<a href="https://colab.research.google.com/github/Hidestament/AtCoder/blob/main/ABC/ABC194.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A - I scream

In [ ]:
a,b = map(int, input().split())

if a+b >= 15 and b >= 8:
  print(1)
elif a+b >= 10 and b >= 3:
  print(2)
elif a+b >= 3:
  print(3)
else:
  print(4)

0 0
4


# B - Job Assignment

## $O(n)の解法$
Aが最小の人とBが最小の人が同じ場合と異なる場合を考えなくてはいけない.

In [30]:
n = int(input())

3


In [31]:
T = [list(map(int, input().split())) for _ in range(n)]
T1 = T[:]
T2 = T[:]

11 7
3 2
6 7


In [32]:
ab_min = sum(min(T, key=lambda x:x[0]+x[1]))
a_min_1 = T1.pop(T1.index(min(T1, key=lambda x:x[0])))[0]
b_min_1 = T1.pop(T1.index(min(T1, key=lambda x:x[1])))[1]

In [ ]:
a_min_2 = T2.pop(T2.index(min(T2, key=lambda x:x[0])))[0]
b_min_2 = T2.pop(T2.index(min(T2, key=lambda x:x[1])))[1]

In [33]:
print(min(max(a_min_1, b_min_1), max(a_min_2, b_min_2), ab_min))

5


## $O(n^2)$, 全列挙で解いてみる.

In [8]:
n = int(input())
a = [0] * n
b = [0] * n
for i in range(n):
  a[i], b[i] = map(int, input().split())

3
8 5
4 4
7 9


In [9]:
ans = 10**6
for i in range(n):
  for j in range(n):
    ans = min(ans, a[i] + b[j] if i == j else max(a[i], b[j]))

In [10]:
print(ans)

5


# C - Squared Error


## 累積和を使う
iを0始まりにして, 固定すると
$$
\sum_{j=1}^{i-1} (A_i - A_j)^2 \\
= \sum_{j=1}^{i-1} \left( A_i^2 - 2 A_i A_j + A_j^2 \right) \\
= i * A_i^2 - 2A_i \sum_{j=1}^{i-1} A_j + \sum_{j=1}^{i-1} A_j^2 
$$
累積和を使えばこれはO(1)で解けるので, iに対してfor文を回す

In [37]:
n = int(input())
a = list(map(int, input().split()))

3
2 8 4


In [86]:
a_j = [0] * (n+1)
a_j_2 = [0] + [0]*n
for i in range(1, n+1):
  a_j[i] = a[i-1] + a_j[i-1]
  a_j_2[i] = a[i-1]**2 + a_j_2[i-1]

In [91]:
ans = 0
for i in range(1, n):
  ans += i * (a[i] ** 2) - 2 * a[i] * a_j[i] + a_j_2[i]

In [92]:
print(ans)

56


## Σの展開を工夫する
$$
\sum_{i=2}^n \sum_{j=1}^{i-1} (A_i - A_j)^2   \\
= \sum_{1 \leq j < i \leq n} (A_i - A_j)^2 \\
= \frac{1}{2}\left( \sum_{i,j=1}^n (A_i - A_j)^2 - \sum_{k=1}^n (A_k - A_k)^2 \right)
$$
最後の変形は, $1\leq j < i \leq n$に対して和をとる事が, $1 \leq i,j \leq n$ に対して和を取ったモノから, $(i,j)$と$(j,i)$で同じモノが2回たされているので, ÷2.
さらに, $i==j の (i,j)$ と同じモノがたされているので, それ分を引いている.

$(A_k -A_k) = 0$より, 
$$
= \frac{1}{2} \sum_{i,j = 1}^n (A_i - A_j) ^2\\
= \frac{1}{2} \left( \sum_{i=1}^n \sum_{j=1}^n A_i^2 - 2 * \left( \sum_{i=1}^n A_i \right) \left( \sum_{j=1}^n A_j \right) + \sum_{i=1}^n \sum_{j=1}^n A_j^2  \right) \\
= \frac{1}{2} \left( n *  \sum_{i=1}^n  A_i^2 - 2 * \left( \sum_{i=1}^n A_i \right)^2  + n *  \sum_{j=1}^n A_j^2  \right) \\

= \frac{1}{2} \left( 2n *  \sum_{i=1}^n  A_i^2 - 2 * \left( \sum_{i=1}^n A_i \right)^2   \right) \\
= \left( n *  \sum_{i=1}^n  A_i^2 -  \left( \sum_{i=1}^n A_i \right)^2   \right) \\
$$

となるので, Aの二乗の和と, 和の二乗を求めれば良い.

In [ ]:
n = int(input())
a = list(map(int, input().split()))

In [ ]:
ans = 0
temp = 0
for i in range(n):
  ans += a[i] ** 2
  temp += a[i]
ans = n * ans - temp**2
print(ans)

# D - Journey
全ての頂点が選ばれたときに連結になる.

よって, 全ての頂点が選ばれる回数の期待値を求めれば良い.

これは, 「ガチャにおいて全ての商品を獲得できる期待値」を求める所謂コンプガチャ


In [1]:
n = int(input())

3


In [5]:
ans = 0
for i in range(1, n):
  ans += 1 / (n-i)
ans *= n

In [6]:
print(ans)